In [ ]:
import wandb
import pandas as pd

run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

In [ ]:
import pandas_profiling

profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

In [ ]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [ ]:
df.info()

In [ ]:
run.finish()